**```
Declare variables
```**

In [ ]:
DECLARE @TGeocodes GeocodesType

**```
Step 1: Copy all the data from Geocodes table into a temp table
```**

In [ ]:
select *
into #tempGeocodes
from Geocodes

**```
Step 2: Trim all spaces in the address column
```**

In [ ]:
update #tempGeocodes
set Address = REPLACE(Address, ' ', '')

**```
Step 3: Copy unique rows into a temp table and drop the previous temp table
```**

In [ ]:
WITH GCodes AS (
    SELECT 
        *, ROW_NUMBER() OVER (PARTITION BY [Address] ORDER BY [Address]) as DuplicateCount
    FROM #tempGeocodes
)
SELECT * 
INTO @TGeocodes
FROM GCodes
WHERE DuplicateCount = 1

DROP TABLE #tempGeocodes

**```
Step 4: Truncate the target table
```**

In [ ]:
TRUNCATE TABLE Geocodes

**```
Insert all the unique rows from the tempt table
```**

In [ ]:
INSERT INTO Geocodes
SELECT [Address], Lat, Lon, LocationType, GeocodeType, CreateDate, LastUpdated
FROM @TGeocodes WHERE DuplicateCount = 1